### Importing R libraries

In [2]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)
# library(mice)
# library(VIM) # for missing data plot

# library(data.table)
# library(Matrix)
# library(caret) # import this before glmnet to avoid rlang version problem
# library(glmnet)
# library(bit64)

# library(slam)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

### Set up and run queries
Do this on Nero GCP as querying from a local computer takes much longer time

In [3]:
# CREDENTIALS depending on LOCATIONS:
# credential <- paste0("/home/", "minh084", "/.config/gcloud/application_default_credentials.json")

# local computer
# credential <- "C:/Users/User/AppData/Roaming/gcloud/application_default_credentials.json"

# Nero onprem
# credential <- "/home/minh084/.config/gcloud/application_default_credentials.json"

# Nero gcp notebook
credential <- "/home/jupyter/.config/gcloud/application_default_credentials.json"

project_id <- "som-nero-phi-jonc101"

Sys.setenv(GOOGLE_APPLICATION_CREDENTIALS = credential)
Sys.setenv(GCLOUD_PROJECT = project_id)
gargle::credentials_app_default()

NULL

In [4]:
library(DBI)
con <- dbConnect(
  bigrquery::bigquery(),
  project = project_id,
  dataset = "shc_core" #, billing = project_id
)
con 
dbListTables(con)

<BigQueryConnection>
  Dataset: som-nero-phi-jonc101.shc_core
  Billing: som-nero-phi-jonc101

[1] "adt"                 "allergy"             "culture_sensitivity"
 [4] "demographic"         "dep_map"             "diagnosis_code"     
 [7] "drg_code"            "encounter"           "family_hx"          
[10] "flowsheet"           "lab_result"          "lda"                
[13] "mapped_meds"         "mar"                 "med_orderset"       
[16] "ndc_code"            "order_med"           "order_proc"         
[19] "order_quest"         "proc_note"           "proc_orderset"      
[22] "procedure_code"      "prov_map"            "social_hx"          
[25] "treatment_team"

### Join with cohort (after removing patients without vital signs)

In [117]:
nrow(labs_clean)
nrow(labs_clean %>% select(anon_id, pat_enc_csn_id_coded) %>% distinct())

cohort <- read.csv("./Data/cohort_final.csv") #41654
nrow(cohort)
labs_clean <- inner_join(cohort, labs_clean) %>% mutate(feature_type = "labs")
nrow(labs_clean)
nrow(labs_clean %>% select(anon_id, pat_enc_csn_id_coded) %>% distinct()) #39226 about 2428 pts have no labs

[1] 1422348

[1] 40838

[1] 41654

Joining, by = c("anon_id", "pat_enc_csn_id_coded")



[1] 1368351

[1] 39226

In [118]:
head(labs_clean)

,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,label_max24,admit_time,admit_label,label_24hr_recent,label_12hr_recent,has_admit_label,first_label,first_label_time_since_admit,acute_to_critical_label,critical_to_acute_label,features,values,result_time,feature_type
,<fct>,<dbl>,<int>,<int>,<fct>,<int>,<int>,<int>,<int>,<int>,<fct>,<int>,<int>,<chr>,<dbl>,<dttm>,<chr>
1,JC29f8ad2,131274729058,40679773,0,2019-08-31 12:52:00+00:00,0,0,0,1,0,0,0,0,"Magnesium, Ser/Plas",2.0,2019-08-31 11:49:00,labs
2,JC29f8ad2,131274729058,40679773,0,2019-08-31 12:52:00+00:00,0,0,0,1,0,0,0,0,"Phosphorus, Ser/Plas",4.1,2019-08-31 11:49:00,labs
3,JC29f8ad2,131274729058,40679773,0,2019-08-31 12:52:00+00:00,0,0,0,1,0,0,0,0,K,6.3,2019-08-31 10:33:00,labs
4,JC29f8ad2,131274729058,40679773,0,2019-08-31 12:52:00+00:00,0,0,0,1,0,0,0,0,K,5.1,2019-08-31 12:45:00,labs
5,JC29f8ad2,131274729058,40679773,0,2019-08-31 12:52:00+00:00,0,0,0,1,0,0,0,0,K,5.7,2019-08-31 11:45:00,labs
6,JC29f8ad2,131274729058,40679773,0,2019-08-31 12:52:00+00:00,0,0,0,1,0,0,0,0,K,6.6,2019-08-31 11:29:00,labs


In [119]:
# save the new lab table, with 9999999 values still available
write.csv(labs_clean, "./Data/labs_clean.csv", row.names=FALSE)

### OLD ---Check overlapping cohort

In [27]:
#llabs is cohort_long_labs
noLabs <- cohort_demo_imputed %>% select(anon_id, pat_enc_csn_id_coded) %>% 
            anti_join(cohort_long_labs %>% select(anon_id, pat_enc_csn_id_coded) %>% distinct()) %>%
            mutate(LABS = "noLabs")
nrow(noLabs) # dropped from 7800 to 348!
head(noLabs, n=1) # 

Joining, by = c("jc_uid", "pat_enc_csn_id_coded")



[1] 348

,jc_uid,pat_enc_csn_id_coded,LABS
,<fct>,<dbl>,<chr>
1,JCde32cd,131238854918,noLabs


In [33]:
# table noLabs noVS show how many are in each and combined
nrow(cohort_demo_imputed)
nrow(noVS)
nrow(noLabs)

VsLabs <- cohort_demo_imputed %>% select(anon_id, pat_enc_csn_id_coded, admit_time, label) %>%
                left_join(noVS) %>% left_join(noLabs)
head(VsLabs)
nrow(VsLabs)
VsLabs %>% count(VS, LABS)

cohort_VsLabs <- VsLabs %>% filter(is.na(VS) | is.na(LABS)) %>% select(-c(VS, LABS))
nrow(cohort_VsLabs)
head(cohort_VsLabs)
cohort_VsLabs %>% count(label)

noVSnoLabs <- VsLabs %>% filter(!is.na(VS) & !is.na(LABS))
nrow(noVSnoLabs)
noVSnoLabs %>% count(label)

[1] 33313

[1] 2316

[1] 348

Joining, by = c("jc_uid", "pat_enc_csn_id_coded")

Joining, by = c("jc_uid", "pat_enc_csn_id_coded")



,jc_uid,pat_enc_csn_id_coded,admit_time,label,VS,LABS
,<fct>,<dbl>,<fct>,<int>,<chr>,<chr>
1,JCcb9495,131066982419,2015-02-13 07:09:00,0,NA,NA
2,JCcb9599,131235926251,2017-08-03 03:50:00,0,NA,NA
3,JCcba5ad,131177568449,2016-02-20 19:27:00,0,NA,NA
4,JCcbfbf7,131193455501,2016-08-21 02:08:00,0,NA,NA
5,JCcc15c6,131254215457,2018-07-09 03:24:00,0,NA,NA
6,JCcc1e6a,131197599694,2016-09-28 03:44:00,0,NA,NA


...

[1] 33053

,jc_uid,pat_enc_csn_id_coded,admit_time,label
,<fct>,<dbl>,<fct>,<int>
1,JCcb9495,131066982419,2015-02-13 07:09:00,0
2,JCcb9599,131235926251,2017-08-03 03:50:00,0
3,JCcba5ad,131177568449,2016-02-20 19:27:00,0
4,JCcbfbf7,131193455501,2016-08-21 02:08:00,0
5,JCcc15c6,131254215457,2018-07-09 03:24:00,0
6,JCcc1e6a,131197599694,2016-09-28 03:44:00,0


label,n
<int>,<int>
0,28493
1,4560


[1] 260

label,n
<int>,<int>
0,229
1,31


In [36]:
nrow(cohort_VsLabs)

[1] 33053

In [34]:
# this is the new cohort, from adjust_cohort to remove those without both VS and LAB values
write.csv(cohort_VsLabs, file = "./Data/cohort_VsLabs.csv", row.names=FALSE)

### Exploration - Labs!!!

In [65]:
slabs %>% filter(ord_value %in% c("N/A","hi") | result_flag == "Abnormal" )

jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
JCe28b26,131179641705,490611891,Glucose by Meter,110,110,70.0,100,mg/dL,,Abnormal,2016-03-26 02:55:00+00:00,2016-03-25 21:52:00+00:00,2016-03-25 21:45:00+00:00,2016-03-25 21:51:00+00:00,0
JCec0b99,131250037071,562669031,Glucose by Meter,265,265,70.0,100,mg/dL,,Abnormal,2018-04-30 01:24:00+00:00,2018-04-29 23:41:00+00:00,2018-04-29 22:25:00+00:00,2018-04-29 23:40:00+00:00,0
JCeb6042,131252026063,567942202,Glucose by Meter,hi,9999999,70.0,100,mg/dL,,Abnormal,2018-07-06 03:38:00+00:00,2018-07-06 01:00:00+00:00,2018-07-06 00:54:00+00:00,2018-07-06 01:00:00+00:00,1
JCcd6008,131140920667,482414867,Glucose by Meter,447,447,70.0,100,mg/dL,,Abnormal,2015-10-27 00:06:00+00:00,2015-10-26 22:19:00+00:00,2015-10-26 22:19:00+00:00,2015-10-26 22:19:00+00:00,0
JCd6f888,131246084077,554641448,Glucose by Meter,349,349,70.0,100,mg/dL,,Abnormal,2018-02-24 01:39:00+00:00,2018-02-23 21:34:00+00:00,2018-02-23 20:35:00+00:00,2018-02-23 21:34:00+00:00,0
JCcff0e8,131066315617,458820624,Glucose by Meter,359,359,70.0,100,mg/dL,,Abnormal,2015-02-07 19:40:00+00:00,2015-02-05 07:45:00+00:00,2015-02-05 02:23:00+00:00,2015-02-05 07:45:00+00:00,0
JCcff0e8,131066315617,458830549,Glucose by Meter,343,343,70.0,100,mg/dL,,Abnormal,2015-02-07 19:40:00+00:00,2015-02-05 14:16:00+00:00,2015-02-05 08:02:00+00:00,2015-02-05 14:15:00+00:00,0
JCdb0e73,131250902180,564976912,Glucose by Meter,470,470,70.0,100,mg/dL,,Abnormal,2018-04-30 07:43:00+00:00,2018-04-30 05:45:00+00:00,2018-04-30 05:28:00+00:00,2018-04-30 05:45:00+00:00,0
JCda3c94,131065279985,457862633,Hemoglobin,N/A,9999999,13.5,17.7,g/dL,,,2015-02-04 15:42:00+00:00,2015-02-04 13:49:00+00:00,2015-02-04 12:54:00+00:00,2015-02-04 13:10:00+00:00,0


In [68]:
slabs %>% filter(ord_num_value == 9999999 & reference_high == 100) # only a few

jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
JCe5dedb,131179853593,490810110,Glucose by Meter,>500,9999999,70,100,mg/dL,,High,2016-02-13 09:45:00+00:00,2016-02-12 23:01:00+00:00,2016-02-12 21:50:00+00:00,2016-02-12 23:00:00+00:00,0
JCd30048,131251139483,565606595,Glucose by Meter,>600,9999999,70,100,mg/dL,,High Panic,2018-06-11 05:48:00+00:00,2018-06-11 02:45:00+00:00,2018-06-11 02:45:00+00:00,2018-06-11 02:41:00+00:00,1
JCd9b8be,131071676672,459602725,Glucose by Meter,>500,9999999,70,100,mg/dL,,,2015-02-05 20:01:00+00:00,2015-02-05 15:03:00+00:00,2015-02-05 14:52:00+00:00,2015-02-05 15:02:00+00:00,0
JCdbb4f1,131255763379,576077570,Glucose by Meter,>600,9999999,70,100,mg/dL,,High Panic,2018-09-01 05:53:00+00:00,2018-09-01 03:41:00+00:00,2018-09-01 03:41:00+00:00,2018-09-01 03:38:00+00:00,1
JCe96c44,131256242428,577099820,Glucose by Meter,>600,9999999,70,100,mg/dL,,High Panic,2018-08-01 02:54:00+00:00,2018-08-01 00:10:00+00:00,2018-08-01 00:10:00+00:00,2018-08-01 00:09:00+00:00,1
JCd1f333,131258175690,581927845,Glucose by Meter,>600,9999999,70,100,mg/dL,,High Panic,2018-10-17 11:44:00+00:00,2018-10-17 08:06:00+00:00,2018-10-17 08:06:00+00:00,2018-10-17 07:52:00+00:00,1
JCeb6042,131252026063,567942522,Glucose by Meter,>600,9999999,70,100,mg/dL,,High Panic,2018-07-06 03:38:00+00:00,2018-07-06 00:58:00+00:00,2018-07-06 00:58:00+00:00,2018-07-06 00:57:00+00:00,1
JCeb6042,131252026063,567946904,Glucose by Meter,>600,9999999,70,100,mg/dL,,High Panic,2018-07-06 03:38:00+00:00,2018-07-06 02:23:00+00:00,2018-07-06 02:23:00+00:00,2018-07-06 02:20:00+00:00,1
JCeb6042,131252026063,567942202,Glucose by Meter,hi,9999999,70,100,mg/dL,,Abnormal,2018-07-06 03:38:00+00:00,2018-07-06 01:00:00+00:00,2018-07-06 00:54:00+00:00,2018-07-06 01:00:00+00:00,1


In [77]:
slabs %>% filter(result_in_range_yn == "Y") %>% count(lab_name)
head(slabs %>% filter(result_in_range_yn == ""), n=10)
slabs %>% filter(result_in_range_yn == "Y" & lab_name == "Magnesium, Ser/Plas")

lab_name,n
<fct>,<int>
"Albumin, Ser/Plas",377
Anion Gap,781
"CO2, Ser/Plas",737
Glucose by Meter,59
Hemoglobin,489
"Magnesium, Ser/Plas",43
Platelet count,697
"Potassium, Ser/Plas",740
"Sodium, Ser/Plas",625


jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
JCcc15c6,131254215457,572191863,"Magnesium, Ser/Plas",2.3,2.3,1.8,2.4,mg/dL,,,2018-07-09 03:24:00+00:00,2018-07-08 23:47:00+00:00,2018-07-08 23:29:00+00:00,2018-07-08 22:22:00+00:00,0
JCcc67c4,131170882153,486368824,"Magnesium, Ser/Plas",1.9,1.9,1.8,2.4,mg/dL,,,2016-01-15 00:38:00+00:00,2016-01-14 23:24:00+00:00,2016-01-14 22:40:00+00:00,2016-01-14 22:32:00+00:00,0
JCccde6e,131259373038,585069536,Glucose by Meter,155,155.0,70.0,200,mg/dL,,,2018-10-10 01:17:00+00:00,2018-10-10 00:09:00+00:00,2018-10-10 00:09:00+00:00,2018-10-10 00:08:00+00:00,0
JCccf3ed,131254647220,573338665,"Magnesium, Ser/Plas",1.7,1.7,1.8,2.4,mg/dL,,Low,2018-07-05 19:23:00+00:00,2018-07-05 16:19:00+00:00,2018-07-05 15:41:00+00:00,2018-07-05 15:19:00+00:00,0
JCccf3ed,131254647220,573347265,Glucose by Meter,187,187.0,70.0,200,mg/dL,,,2018-07-05 19:23:00+00:00,2018-07-05 16:33:00+00:00,2018-07-05 16:33:00+00:00,2018-07-05 15:55:00+00:00,0
JCcf49e4,131253970652,571581261,Glucose by Meter,190,190.0,70.0,200,mg/dL,,,2018-06-15 00:36:00+00:00,2018-06-14 22:40:00+00:00,2018-06-14 22:40:00+00:00,2018-06-14 22:31:00+00:00,0
JCcf5064,131175826496,487351180,"Magnesium, Ser/Plas",1.8,1.8,1.8,2.4,mg/dL,,,2016-01-05 12:49:00+00:00,2016-01-05 09:23:00+00:00,2016-01-05 08:27:00+00:00,2016-01-05 08:47:00+00:00,0
JCd00bd4,131193224666,503347753,"Magnesium, Ser/Plas",2.2,2.2,1.8,2.4,mg/dL,,,2016-07-05 03:12:00+00:00,2016-07-04 23:29:00+00:00,2016-07-04 22:28:00+00:00,2016-07-04 23:00:00+00:00,0
JCd02348,131253461651,570735311,"Magnesium, Ser/Plas",2.4,2.4,1.8,2.4,mg/dL,,,2018-06-07 05:00:00+00:00,2018-06-07 02:31:00+00:00,2018-06-07 01:57:00+00:00,2018-06-07 02:00:00+00:00,0


jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
JCcf45a6,131067176155,459383639,"Magnesium, Ser/Plas",2.0,2.0,1.8,2.4,mg/dL,Y,,2015-01-06 23:55:00+00:00,2015-01-06 22:13:00+00:00,2015-01-06 21:47:00+00:00,2015-01-06 20:10:00+00:00,0
JCe3251e,131073240555,460864479,"Magnesium, Ser/Plas",1.9,1.9,1.8,2.4,mg/dL,Y,,2015-03-11 04:55:00+00:00,2015-03-11 03:55:00+00:00,2015-03-11 03:10:00+00:00,2015-03-11 03:20:00+00:00,0
JCd3d91f,131064921306,457567908,"Magnesium, Ser/Plas",1.8,1.8,1.8,2.4,mg/dL,Y,,2015-01-17 19:03:00+00:00,2015-01-17 17:41:00+00:00,2015-01-17 16:58:00+00:00,2015-01-17 17:10:00+00:00,0
JCe2cb29,131073469706,461026392,"Magnesium, Ser/Plas",2.0,2.0,1.8,2.4,mg/dL,Y,,2015-02-07 06:10:00+00:00,2015-02-07 05:05:00+00:00,2015-02-07 02:58:00+00:00,2015-02-07 04:25:00+00:00,0
JCe9d59f,131069240552,459502316,"Magnesium, Ser/Plas",1.8,1.8,1.8,2.4,mg/dL,Y,,2015-02-07 10:09:00+00:00,2015-02-07 08:14:00+00:00,2015-02-07 07:19:00+00:00,2015-02-07 04:50:00+00:00,0
JCd362dc,131067130693,459676324,"Magnesium, Ser/Plas",1.8,1.8,1.8,2.4,mg/dL,Y,,2015-01-22 22:42:00+00:00,2015-01-22 17:41:00+00:00,2015-01-22 16:55:00+00:00,2015-01-22 17:12:00+00:00,0
JCd362dc,131067130693,459413059,"Magnesium, Ser/Plas",1.9,1.9,1.8,2.4,mg/dL,Y,,2015-01-22 22:42:00+00:00,2015-01-18 14:44:00+00:00,2015-01-18 06:01:00+00:00,2015-01-18 14:06:00+00:00,0
JCd362dc,131067130693,459440094,"Magnesium, Ser/Plas",1.8,1.8,1.8,2.4,mg/dL,Y,,2015-01-22 22:42:00+00:00,2015-01-18 18:03:00+00:00,2015-01-18 16:49:00+00:00,2015-01-18 17:00:00+00:00,0
JCdb3b8c,131067095683,459315521,"Magnesium, Ser/Plas",2.2,2.2,1.8,2.4,mg/dL,Y,,2015-01-04 06:42:00+00:00,2015-01-04 02:11:00+00:00,2015-01-04 01:25:00+00:00,2015-01-04 01:42:00+00:00,0


In [63]:
options(repr.matrix.max.rows=900, repr.matrix.max.cols=20)
slabs %>% filter(ord_num_value == 9999999) %>% group_by(lab_name) %>% mutate(n=n()) %>% arrange(n)

jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label,n
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>
JCda3c94,131065279985,457862633,Hemoglobin,N/A,9999999,13.5,17.7,g/dL,,,2015-02-04 15:42:00+00:00,2015-02-04 13:49:00+00:00,2015-02-04 12:54:00+00:00,2015-02-04 13:10:00+00:00,0,1
JCe58f3a,131066472308,458825682,"Creatinine, Ser/Plas",<0.1,9999999,NA,,mg/dL,,,2015-01-08 09:38:00+00:00,2015-01-08 05:55:00+00:00,2015-01-08 05:03:00+00:00,2015-01-08 05:00:00+00:00,0,1
JCeaafe3,131066983116,459235759,Platelet count,N/A,9999999,150.0,400,K/uL,,,2015-02-07 07:47:00+00:00,2015-02-07 05:45:00+00:00,2015-02-07 04:55:00+00:00,2015-02-07 05:00:00+00:00,0,3
JCdb4266,131066053542,458448054,Platelet count,N/A,9999999,150.0,400,K/uL,,,2015-01-31 06:42:00+00:00,2015-01-31 04:15:00+00:00,2015-01-31 01:47:00+00:00,2015-01-31 03:36:00+00:00,0,3
JCdc4b08,131065650599,458134339,Platelet count,N/A,9999999,150.0,400,K/uL,,,2015-02-10 21:17:00+00:00,2015-02-10 17:34:00+00:00,2015-02-10 14:22:00+00:00,2015-02-10 16:20:00+00:00,0,3
JCd92132,131249534245,561316092,WBC,<0.1,9999999,4.0,11.0,K/uL,,Low Panic,2018-04-19 01:32:00+00:00,2018-04-18 23:39:00+00:00,2018-04-18 22:53:00+00:00,2018-04-18 23:13:00+00:00,1,10
JCebf793,131250683133,564432029,WBC,<0.1,9999999,4.0,11.0,K/uL,,Low Panic,2018-04-22 05:02:00+00:00,2018-04-22 02:50:00+00:00,2018-04-22 01:06:00+00:00,2018-04-22 01:32:00+00:00,0,10
JCd45820,131194532027,504496090,WBC,<0.1,9999999,4.0,11.0,K/uL,,Low Panic,2016-09-14 10:27:00+00:00,2016-09-14 09:25:00+00:00,2016-09-14 09:05:00+00:00,2016-09-14 09:07:00+00:00,0,10
JCd0731e,131199578754,508186098,WBC,<0.1,9999999,4.0,11.0,K/uL,,Low Panic,2016-09-09 12:02:00+00:00,2016-09-09 09:42:00+00:00,2016-09-09 08:55:00+00:00,2016-09-09 09:09:00+00:00,0,10


In [79]:
slabs %>% count(ord_value) %>% arrange(ord_value)

ord_value,n
<fct>,<int>
,534
<0.1,22
<0.2,28
<1.0,28
<10,405
<15,57
<2,81
<2.0,11
<20,4


In [84]:
head(slabs %>% count(ord_num_value) %>% arrange(desc(ord_num_value)))
tail(slabs %>% count(ord_num_value) %>% arrange(desc(ord_num_value)))
slabs %>% filter(ord_num_value == 6589)

ord_num_value,n
<dbl>,<int>
9999999,889
6589,1
6307,1
5985,1
5580,1
5336,1


ord_num_value,n
<dbl>,<int>
0.06,1
0.04,2
0.03,1
0.02,1
0.01,2
NA,534


jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
JCe53597,131238056628,538257363,"AST (SGOT), Ser/Plas",6589,6589,NA,<40,U/L,,High,2017-08-17 23:06:00+00:00,2017-08-17 21:01:00+00:00,2017-08-17 19:02:00+00:00,2017-08-17 19:37:00+00:00,1


In [86]:
head(slabs %>% filter(result_flag == ""), n=20)

jc_uid,pat_enc_csn_id_coded,order_id_coded,lab_name,ord_value,ord_num_value,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,admit_time,result_time_jittered_utc,order_time_jittered_utc,taken_time_jittered_utc,label
<fct>,<dbl>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>
JCcc15c6,131254215457,572191863,"Magnesium, Ser/Plas",2.3,2.3,1.8,2.4,mg/dL,,,2018-07-09 03:24:00+00:00,2018-07-08 23:47:00+00:00,2018-07-08 23:29:00+00:00,2018-07-08 22:22:00+00:00,0
JCcc67c4,131170882153,486368824,"Magnesium, Ser/Plas",1.9,1.9,1.8,2.4,mg/dL,,,2016-01-15 00:38:00+00:00,2016-01-14 23:24:00+00:00,2016-01-14 22:40:00+00:00,2016-01-14 22:32:00+00:00,0
JCccde6e,131259373038,585069536,Glucose by Meter,155,155.0,70.0,200,mg/dL,,,2018-10-10 01:17:00+00:00,2018-10-10 00:09:00+00:00,2018-10-10 00:09:00+00:00,2018-10-10 00:08:00+00:00,0
JCccf3ed,131254647220,573347265,Glucose by Meter,187,187.0,70.0,200,mg/dL,,,2018-07-05 19:23:00+00:00,2018-07-05 16:33:00+00:00,2018-07-05 16:33:00+00:00,2018-07-05 15:55:00+00:00,0
JCcf49e4,131253970652,571581261,Glucose by Meter,190,190.0,70.0,200,mg/dL,,,2018-06-15 00:36:00+00:00,2018-06-14 22:40:00+00:00,2018-06-14 22:40:00+00:00,2018-06-14 22:31:00+00:00,0
JCcf5064,131175826496,487351180,"Magnesium, Ser/Plas",1.8,1.8,1.8,2.4,mg/dL,,,2016-01-05 12:49:00+00:00,2016-01-05 09:23:00+00:00,2016-01-05 08:27:00+00:00,2016-01-05 08:47:00+00:00,0
JCd00bd4,131193224666,503347753,"Magnesium, Ser/Plas",2.2,2.2,1.8,2.4,mg/dL,,,2016-07-05 03:12:00+00:00,2016-07-04 23:29:00+00:00,2016-07-04 22:28:00+00:00,2016-07-04 23:00:00+00:00,0
JCd02348,131253461651,570735311,"Magnesium, Ser/Plas",2.4,2.4,1.8,2.4,mg/dL,,,2018-06-07 05:00:00+00:00,2018-06-07 02:31:00+00:00,2018-06-07 01:57:00+00:00,2018-06-07 02:00:00+00:00,0
JCd0c9f7,131235974855,532986715,"Magnesium, Ser/Plas",2.1,2.1,1.8,2.4,mg/dL,,,2017-06-27 08:26:00+00:00,2017-06-27 06:41:00+00:00,2017-06-27 04:46:00+00:00,2017-06-27 05:31:00+00:00,0
